In [1]:
# import libraries
import numpy as np
import pandas as pd
from pandas import ExcelWriter
import time
from datetime import datetime
import datetime as dt
import warnings
warnings.simplefilter("ignore")

In [2]:
# output file name and location
current_date = time.strftime("%Y%m%d")
output_file = 'natural_persons_main' + current_date + '.xlsx'

In [3]:
# import data
file_path = 'report_2.xlsx'
df = pd.read_excel(file_path)

***

In [ ]:
# filter for individuals
df_natural = df[df['Party Type']=='Natural Person']

In [4]:
#Preprocessing party name
df_natural.loc[:,'Party Name_pre']=df_natural['Party Name'].copy()
df_natural['Party Name_pre'] = df_natural['Party Name_pre'].str \
                                                           .lower() \
                                                           .replace(r'[^\w\s]',"")

In [ ]:
#Prepocessing 'Identification Document Number' remove space & casing
df_natural.loc[:,'Identification Document Number_pre']=df_natural['Identification Document Number'].copy()
df_natural['Identification Document Number_pre'] = df_natural['Identification Document Number_pre'].str \
                                                                                                   .lower() \
                                                                                                   .replace(r'[^\w\s]',"")

In [ ]:
# create beneficiary only list
bene_only = df_natural.groupby('Party ID')['Relationship (Party Role)'] \
                      .agg(set) \
                      .reset_index(name = 'role_set')

bene_only = bene_only[bene_only['role_set'] == {'Beneficiary'}]
bene_only_list = bene_only['Party ID'].to_list()

df_natural['bene_only'] = df_natural['Party ID'].isin(bene_only_list)

In [ ]:
#dates
df_natural['Date of Birth']  = pd.to_datetime(df_natural['Date of Birth'], format='%Y-%m-%d', errors='coerce').dt.date

***

### HK ID without HK Passport or DON

In [ ]:
# want check
# add new column:
# Party only has HK nationality OR Party has dual nationality

In [5]:
tab25 = df_natural.copy()

# create a key
tab25['key'] = tab25['Party Name_pre'].astype(str) + tab25['Date of Birth'].astype(str)

# only HK Nationality
tab25 = tab25[tab25["Nationality"] == "Hong Kong"]

# only HK documents
tab25 = tab25[tab25["Identification Document Issue Country"] == "Hong Kong"]

# set of all ID documents a party has
tab25_id = tab25.groupby('key')['Identification Document Type'] \
                .agg(set) \
                .reset_index(name='id_set')

id_set = tab25_id['id_set'].tolist()

def check_id(id_set): # function to check that id set does not contain HK passport
    hk_set = {'Passport, Declaration of Nationality'}
    result = id_set.intersection(hk_set)
    return len(result) == 0 #if TRUE: empty intersection. means HK Passport not present

tab25_id['passport_missing'] = tab25_id['id_set'].apply(lambda x: check_id(x))
tab25_id = tab25_id[tab25_id['passport_missing'] == True]

# compare keep list to original table
tab25_id_list = tab25_id['key'].tolist()
tab25['exist'] = tab25['key'].isin(tab25_id_list)
tab25 = tab25[tab25['exist'] == True]

# remove duplicates and sort
tab25 = tab25.drop_duplicates(subset = ['Party ID', 'key']) \
             .sort_values(by = ["key",'Party ID'])

# select columns
tab25_columns = tab25[['Party ID','Party Name',
                       'RM UID.1','RM Name.1',
                       'Date of Birth','Nationality',
                       'Identification Document Type',
                       'Identification Document Issue Country',
                       'Identification Document Number',
                       'Identification Date of Issue',
                       'Identification Document Expiry Date',
                       'Identification Document - Please Specify',
                       'Override Expiry',
                       'Override Expiry Reason',
                       'Override Expiry Justification',
                       'Deceased Flag',
                       'bene_only']]

In [6]:
tab25_columns.to_csv('HK ID no Passport or DON.xlsx', index=False)